In [1]:
import pandas as pd
import powerplantmatching as pm
import pypsa
import pycountry
import os 
import time

ERROR 1: PROJ: proj_create_from_database: Open of /trinity/home/thomgeo/micromamba/envs/open_eraa/share/proj failed


In [2]:
def build_inflows(inflows):

    base_year_hydro = pd.Series(
    inflows.index.levels[3].astype(int), 
    inflows.index.levels[3]).subtract(year).abs().idxmin()

    inflows = inflows.loc[:, str(climate_year),:, base_year_hydro,:]
    
    inflow_grouper = pd.Series(["hydro", "ROR", "PHS Open", "pondage"],inflows.index.levels[2])
    
    #inflow_grouper.to_csv('commodity_filtered.csv', index=False)

    inflows = inflows.unstack(2).groupby(inflow_grouper, axis=1).sum().multiply(1) 
    inflows = inflows.unstack(0).stack(0)
    
    inflows.index = [" ".join(i) for i in inflows.index]
    
    inflows = inflows.T
    
    inflows.index = snapshots  
   
    return inflows


In [3]:
def add_existing_storage():  
 
    storage = capacity.loc[["PHS", "reservoir", "battery"]].unstack().reset_index().copy()
    storage.columns = ["bus", "carrier", "p_nom"]
    storage.index = storage.bus + " " + storage.carrier

    p_min_pu = capacity.loc[["PHS (pumping)", "reservoir (pumping)",]].div(capacity.loc[["PHS", "reservoir"]].add(1e-5).values)
    p_min_pu.index = ["PHS", "reservoir"]

    storage["p_min_pu"] = (
        p_min_pu.unstack()
        .reindex(pd.MultiIndex.from_arrays([storage.bus, storage.carrier]))
    ).values

    storage.loc[storage.carrier=="battery", "p_min_pu"] = -1

    storage = storage[storage.p_nom >0]

    storage_inflows = inflows[storage.loc[storage.carrier=="reservoir"].index]

    storage_capacity_raw = pd.read_excel(excel_file, sheet, index_col=1, header=28).dropna(how="all", axis=1)

    storage_capacity= storage_capacity_raw.groupby(
        ["ROR", "reservoir", "reservoir", "PHS", "battery"]
    ).sum()

    storage["max_hours"] = storage_capacity.unstack().reindex(
        pd.MultiIndex.from_arrays(
            [storage.bus, storage.carrier]  
        )
    ).div(
        storage.p_nom.values
    ).values

    map_storage = pd.Series( ["battery inverter", "PHS", "hydro"], ["battery", "PHS", "reservoir"])

    storage["efficiency_store"] = technology_data.loc[:, "efficiency", :].reindex(storage.carrier.map(map_storage)).value.values

    n.madd(
        "StorageUnit",
        storage.index,
        **storage,
        inflow=storage_inflows,
        invest_status = "existing"
    )

In [4]:
def add_existing_storage(all_c):  
 
 
    hydro = pd.read_csv("data/Dashboard_raw_data/Hydro additional information.csv",header=0)
    
    all_c = all_c.reset_index(name="Value")
    
    closest_year = all_c.loc[(all_c['Target year'] - base_year).abs().idxmin(), 'Target year']

    all_cy = all_c[all_c['Target year'] == closest_year]
    
    #all_cy.to_csv('all_cy.csv', index=False)

    all_cdata = (all_cy
      .pivot(index="Technology", columns="Market_Node", values=all_cy.columns[-1]) 
      )

    storage = all_cdata.loc[["PHS","PHS Open","hydro","pondage","battery"]].unstack().reset_index().copy()
    storage.columns = ["bus", "carrier", "p_nom"]
    storage.index = storage.bus + " " + storage.carrier

    hydro = hydro[hydro.apply(lambda row: row.astype(str).str.contains("ERAA 2025 pre-CfE").any(), axis=1)]
    #hydro.to_csv('hydro.csv', index=False)

    
    target_year = hydro.loc[hydro["TARGET_YEAR"].sub(base_year).abs().idxmin(), "TARGET_YEAR"]
    hydro_y = hydro[hydro["TARGET_YEAR"] == target_year]
    
    #hydro_y.to_csv('hydroyear.csv', index=False)
    
    hydro_pump = -(
    hydro_y.loc[
        hydro_y["PEMMDB_PLANT_TYPE"].isin(["Closed loop pumping", "Open loop pumping"])
    ]
    .pivot(index="PEMMDB_PLANT_TYPE", columns="MARKET_NODE", values="MAX PUMPING CAP (MW)")
    .fillna(0)
    .rename(index={
        "Closed loop pumping": "PHS",
        "Open loop pumping": "PHS Open"
    })
)

    hydro_pump=hydro_pump.drop(columns=["UA00"], errors='ignore').loc[:, (hydro_pump != 0).any(axis=0)]
    #hydro_pump.to_csv('hydrostorage.csv', index=False)      

    hydro_cap=all_cdata.loc[all_cdata.index.isin(["PHS", "PHS Open"])]
    
    #hydro_cap.loc[:, hydro_cap.notna().any(axis=0)].to_csv('hydrocapacity.csv', index=False)

    p_min_pu = hydro_pump.div(hydro_cap.loc[:, hydro_cap.notna().any(axis=0)].add(1e-5).values).fillna(0)
    p_min_pu.index = ["PHS", "PHS Open"]
    
    p_min_pu.to_csv('p_min_pu.csv', index=False)
    
    valid_countries = p_min_pu.columns[(p_min_pu != 0).any(axis=0)]
    p_min_pu_filtered = p_min_pu[valid_countries]
    p_min_pu_stacked = p_min_pu_filtered.stack().fillna(0)  # MultiIndex: (carrier, bus)
    storage_index = pd.MultiIndex.from_arrays([storage.carrier, storage.bus])
    storage["p_min_pu"] = p_min_pu_stacked.reindex(storage_index).values 
    
    #storage.to_csv('storage.csv', index=False)

    storage.loc[storage.carrier=="battery", "p_min_pu"] = -1
    storage.loc[storage.carrier=="hydro", "p_min_pu"] = 0
    storage.loc[storage.carrier=="pondage", "p_min_pu"] = 0

    storage = storage[storage.p_nom >0]
    
    #storage.to_csv('storage.csv', index=False)
    
    storage_inflows = inflows[storage.loc[storage.carrier=="PHS Open"].index]
    
    #storage_inflows .to_csv('storage_inflows0.csv', index=False)
    
    storage_inflows = pd.concat([storage_inflows,inflows[storage.loc[storage.carrier == "hydro"].index]],axis=1)
    storage_inflows = pd.concat([storage_inflows,inflows[storage.loc[storage.carrier == "pondage"].index]],axis=1)
    
    storage_inflows.to_csv('storage_inflows.csv', index=False)
    
    hydro_stored = 1000000*(hydro_y.loc[hydro_y["PEMMDB_PLANT_TYPE"].isin(["Closed loop pumping", "Open loop pumping", "Reservoir", "Pondage"])]
      .pivot(index="PEMMDB_PLANT_TYPE", columns="MARKET_NODE", values="Storage Capacity [TWh]")
      .fillna(0)
      .rename(index={
          "Closed loop pumping": "PHS",
          "Open loop pumping": "PHS Open",
          "Reservoir": "hydro",
          "Pondage": "pondage"                    
      })
)       

    print(hydro_stored)
        
    battery = pd.read_csv("data/Dashboard_raw_data/Batteries additional information.csv",header=0)
    battery = battery[battery.apply(lambda row: row.astype(str).str.contains("ERAA 2025 pre-CfE").any(), axis=1)]
    battery.to_csv('battery.csv', index=False)
    
    battery_y = battery[battery["TARGET_YEAR"] == base_year]
    
    battery_y.to_csv('battery_y.csv', index=False)
    
    battery_by_node = battery_y.groupby("MARKET_NODE")["STORAGE CAPACITY (MWh)"].sum().reset_index()
    
    battery_by_node.to_csv('battery_y.csv', index=False)
    battery_by_node["Technology"] = "battery"
    battery_pivot = battery_by_node.pivot(index="Technology", columns="MARKET_NODE", values="STORAGE CAPACITY (MWh)").fillna(0)
    
    battery_pivot.to_csv('battery_pivot.csv', index=False)
    
    storage_capacity=pd.concat([hydro_stored, battery_pivot])
    
    storage_capacity.to_csv('storage_capacity.csv', index=False)
      
    valid_countries = storage_capacity.columns[(storage_capacity != 0).any(axis=0)]
    stored_filtered = storage_capacity[valid_countries]
        
    stored_stacked = stored_filtered.stack() 
    
    storage_index = pd.MultiIndex.from_arrays([storage.carrier, storage.bus])
    
    #Data in #.## TWh, so 5000 MWh as minimum
    storage["max_hours"] = stored_stacked.reindex(storage_index).clip(lower=5000).div(storage.p_nom.values).values 
       
    map_storage = pd.Series( ["battery inverter", "PHS", "hydro","hydro","hydro"], ["battery", "PHS", "PHS Open","hydro","pondage"])

    storage["efficiency_store"] = technology_data.loc[:, "efficiency", :].reindex(storage.carrier.map(map_storage)).value.values
    
    storage.to_csv('storage.csv', index=False)

    n.add(
        "StorageUnit",
        storage.index,
        **storage,
        inflow=storage_inflows,
        invest_status = "existing"
    )

In [5]:
def prepare_commodity_prices(commodity_prices):
    
    fuels = ["co2", "gas", "coal", "oil","lignite", "hydrogen", "nuclear"]
    C02= [57,0, 94, 0,101, 0, 85.3] #gas, co2, coal, hydrogen, lignite, nuclear, oil
        
    def assign_fuel_group(fuel_name):
      fuel_name_lower = str(fuel_name).lower()
      for f in fuels:
          if f.lower() in fuel_name_lower:
              return "CCGT" if f.lower() == "gas" else f
      return None
    
    commodity_prices["Fuel"] = commodity_prices["Fuel"].apply(assign_fuel_group)  
    
    commodity_prices_filtered = commodity_prices[commodity_prices["Fuel"].notna()]
    
    commodity_prices["Fuel"] = commodity_prices["Fuel"].replace("gas", "CCGT")
    
    #commodity_prices.to_csv('commodity_filteredFuel.csv', index=False)
    
    commodity_prices = (
    commodity_prices_filtered
    .groupby(["Year", "Fuel"], as_index=False)["Value"]
    .mean()
    )

     
    commodity_prices = commodity_prices.pivot(index="Fuel", columns="Year", values="Value").reset_index()
     
    commodity_prices["CO2"] = C02
     
    #commodity_prices.to_csv('commodity_column.csv', index=False) 
       
    #commodity_prices_filtered.to_csv('commodity_filtered.csv', index=False)  
    
    commodity_prices = commodity_prices.set_index("Fuel")
    
    commodity_prices = commodity_prices.apply(pd.to_numeric, errors="coerce")
      
    converter = pd.DataFrame(3.6 , commodity_prices.index, commodity_prices.columns)
    converter["CO2"] = 3.6/1000 
    converter.loc["co2"] = 1

    commodity_prices = converter*commodity_prices


    to_add = commodity_prices.reindex(["CCGT", "CCGT"])
    to_add.index = ["OCGT", "biomass"]
    #to_add.loc[["other"]] = 0
    to_add.loc[["biomass"]] = biomass_price
    to_add.loc[["biomass"], "CO2"] = 0
    commodity_prices = pd.concat([commodity_prices, to_add])
    commodity_prices.to_csv('commodity_prices.csv', index=False)
    
    return commodity_prices

def add_renewables():
       
    res = capacity.loc[["CSP","CSPS","onwind","offwind","PVfixed","PVIroof", "PVRroof", "PVtrack","ROR"]].unstack().copy()
    res = res.reset_index()
    res.columns = ["bus", "carrier", "p_nom"]
    res.index = res.bus + " " + res.carrier

    res = res[res.p_nom>0]
    
    vre=["CSP","CSP-stor","onwind","offwind","solar-fix","solar-rsd", "solar-ind","solar-track"]
   
    p_max_pu = pd.concat(
        [pd.read_hdf(snakemake.input.res_profile, tech).loc[:, "WS{:02}".format(climate_year), str(base_year), :] for tech in vre],
        axis=1
    )
    
    #print("found RES in res_profile")
    
    p_max_pu.columns = vre
    p_max_pu = p_max_pu.unstack(0).stack(0)
    p_max_pu.index = [" ".join(i) for i in p_max_pu.index]
    p_max_pu.columns = snapshots
    p_max_pu = p_max_pu.T

    #print(len(inflows[res.filter(like="ROR", axis=0).index]))
    #print(len(res.filter(like="ROR", axis=0).p_nom))

    p_max_pu = pd.concat(
        [p_max_pu, inflows[res.filter(like="ROR", axis=0).index].div(res.filter(like="ROR", axis=0).p_nom).clip(upper=1)],
        axis=1
    )
    
    #p_max_pu = pd.concat(
    #    [p_max_pu, inflows[res.filter(like="pondage", axis=0).index].div(res.filter(like="pondage", axis=0).p_nom)],
    #    axis=1
    #)
    
    p_max_pu[res.index].to_csv('res.csv', index=False)

    n.add(
        "Generator",
        res.index,
        **res,
        p_max_pu = p_max_pu[res.index],
        invest_status="policy"
    )
    
def add_dispatchables():
         
    plants = dispatchable_plants.query("(entry <= @year) & exit > @year")

    plants["marginal_cost"] = (
        commodity_prices
        .reindex(plants.carrier)[base_year].div(
            plants.efficiency.values,
        ).add(
            (
                commodity_prices.reindex(
                    plants.carrier)["CO2"]
                .multiply(
                    commodity_prices.loc["co2", base_year]
                )
            )
        ).add(plants.var_om.values).values
    )
    #plants.loc[:, "p_min_pu"] = 0
    plants.loc[:, "committable"]= True
    #plants.loc[:, "up_time_before"] = 2
    #plants.loc[:, "down_time_before"] = 100
    
    
    plants.to_csv('plants.csv', index=False)
    n.add(
        "Generator",
        plants.index,
        **plants,
    )

def group_luxembourg(demand, links):
    
    demand_grouper = pd.Series(demand.columns, demand.columns)
    demand_grouper.loc[demand_grouper.index.str[:2] == "LU"] = "LUG1"
    demand = demand.groupby(demand_grouper, axis=1).sum()

    links.loc[links.bus0.str[:2] == "LU", "bus0"] = "LUG1"
    links.loc[links.bus1.str[:2] == "LU", "bus1"] = "LUG1"
    
    return demand, links

def add_dsr():
    
    
    dsr=pd.read_hdf(snakemake.input.dsr)
    
    dsr = dsr.loc[base_year, :]
    
    dsr.unstack(1).columns = ["bus", "p_nom", "marginal_cost"]
    
    dsr=dsr.drop(columns=["hours","TARGET_YEAR"])
    
    dsr["carrier"] = "DSR"
    
    dsr.index = [f"{bus} {carrier} {i}" for i, (bus, carrier) in enumerate(zip(dsr.bus, dsr.carrier))]

    
#    dsr.index = [dsr.bus + " " + dsr.carrier + " ".join(i) for i in len(inflows.index)] 
    
    dsr.to_csv('DSR.csv', index=False)
    
    print(dsr.index)

    n.add(
        "Generator",
        dsr.index,
        **dsr,
        invest_status = "existing"
    )
    

def set_investment_bounds():
    
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_max"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]
    n.generators.loc[n.generators.invest_status == "policy", "p_nom_min"] = n.generators.loc[n.generators.invest_status == "policy", "p_nom"]
    

In [6]:
commodity_prices_raw = pd.read_csv("data/Dashboard_raw_data/Commodity Prices.csv",header=0)

all_cap=pd.read_hdf("resources/all_capacities.h5")

year = int(2030)
climate_year = 1

In [7]:
years = range(2028, 2035)

#save_path = snakemake.output.network

biomass_price = 7# snakemake.config["biomass_price"]

dispatchable = ["CCGT", "OCGT", 'biomass', 'coal', 'lignite', 'nuclear','oil','hydrogen']
distributed_resources = ["onwind", "offwind", "CSP","solar", "battery"]

technologies_for_investment = ["OCGT", "CCGT"]

inflows_raw = pd.read_hdf("resources/inflow.h5", "inflow")

snapshots = pd.date_range(start="2010-01-01", freq="h", periods=8760)#.strftime('%m-%d %H:%M:%S')

base_year = pd.Series(
all_cap.index.levels[0].astype(int), 
all_cap.index.levels[0]).subtract(year).abs().idxmin()

In [8]:
technology_data = pd.read_csv("technology-data/outputs/costs_2025.csv", index_col=[0,1])
demand = pd.read_hdf("resources/demand.h5")

demand = demand.loc[:, "WS{:02}".format(climate_year), :, str(base_year), :].unstack(1)
demand.index = snapshots

In [9]:
links = pd.read_hdf("resources/ntcs.h5", "p_nom")
links = links[base_year].unstack(1)
links = links.iloc[:len(snapshots)]

links_p_max_pu = pd.read_hdf("resources/ntcs.h5", "p_max_pu")
links_p_max_pu = links_p_max_pu[base_year].unstack(2)
links_p_max_pu = links_p_max_pu.iloc[:len(snapshots)]
links_p_max_pu.index = snapshots
links.dropna(inplace=True)
links_p_max_pu.dropna(axis=1, inplace=True)
links["carrier"] = [i[-2:] for i in links.index]

In [10]:
demand, links = group_luxembourg(demand, links)

commodity_prices = prepare_commodity_prices(commodity_prices_raw[commodity_prices_raw.apply(lambda row: row.astype(str).str.contains("ERAA 2025 post-CfE").any(), axis=1)])

dispatchable_plants = pd.read_hdf("resources/dispatchable_capacities.h5")

/tmp/ipykernel_1750180/862999074.py:139: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/tmp/ipykernel_1750180/862999074.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/tmp/ipykernel_1750180/862999074.py:17: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [11]:
n = pypsa.Network()
n.set_snapshots(snapshots)

capacity=pd.read_hdf("resources/investcap.h5")[base_year].unstack(1)

buses = (
    capacity.sum()[capacity.abs().sum() >0].index
    .union(demand.columns)
    .union(links.bus0.unique())
    .union(links.bus1.unique())
)

pd.Series(buses).to_csv('buses.csv', index=False)

n.add(
    "Bus", 
    buses, 
    carrier = "electricity", 
    country = buses.str[:2]
)

n.add(
    "Load", 
    demand.columns,
    bus=demand.columns,
    p_set = demand
)

inflows = build_inflows(inflows_raw)

/tmp/ipykernel_1750180/4291473729.py:13: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/tmp/ipykernel_1750180/4291473729.py:14: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [14]:
all_c = all_cap.copy()

In [16]:
#def add_existing_storage(all_c):  


hydro = pd.read_csv("data/Dashboard_raw_data/Hydro additional information.csv",header=0)

all_c = all_c.reset_index(name="Value")

closest_year = all_c.loc[(all_c['Target year'] - base_year).abs().idxmin(), 'Target year']

all_cy = all_c[all_c['Target year'] == closest_year]

#all_cy.to_csv('all_cy.csv', index=False)

all_cdata = (all_cy
  .pivot(index="Technology", columns="Market_Node", values=all_cy.columns[-1]) 
  )

storage = all_cdata.loc[["PHS","PHS Open","hydro","pondage","battery"]].unstack().reset_index().copy()
storage.columns = ["bus", "carrier", "p_nom"]
storage.index = storage.bus + " " + storage.carrier

hydro = hydro[hydro.apply(lambda row: row.astype(str).str.contains("ERAA 2025 pre-CfE").any(), axis=1)]
#hydro.to_csv('hydro.csv', index=False)


target_year = hydro.loc[hydro["TARGET_YEAR"].sub(base_year).abs().idxmin(), "TARGET_YEAR"]
hydro_y = hydro[hydro["TARGET_YEAR"] == target_year]

#hydro_y.to_csv('hydroyear.csv', index=False)

hydro_pump = -(
hydro_y.loc[
    hydro_y["PEMMDB_PLANT_TYPE"].isin(["Closed loop pumping", "Open loop pumping"])
]
.pivot(index="PEMMDB_PLANT_TYPE", columns="MARKET_NODE", values="MAX PUMPING CAP (MW)")
.fillna(0)
.rename(index={
    "Closed loop pumping": "PHS",
    "Open loop pumping": "PHS Open"
})
)

hydro_pump=hydro_pump.drop(columns=["UA00"], errors='ignore').loc[:, (hydro_pump != 0).any(axis=0)]
#hydro_pump.to_csv('hydrostorage.csv', index=False)      

hydro_cap=all_cdata.loc[all_cdata.index.isin(["PHS", "PHS Open"])]

#hydro_cap.loc[:, hydro_cap.notna().any(axis=0)].to_csv('hydrocapacity.csv', index=False)

p_min_pu = hydro_pump.div(hydro_cap.loc[:, hydro_cap.notna().any(axis=0)].add(1e-5).values).fillna(0)
p_min_pu.index = ["PHS", "PHS Open"]

p_min_pu.to_csv('p_min_pu.csv', index=False)

valid_countries = p_min_pu.columns[(p_min_pu != 0).any(axis=0)]
p_min_pu_filtered = p_min_pu[valid_countries]
p_min_pu_stacked = p_min_pu_filtered.stack().fillna(0)  # MultiIndex: (carrier, bus)
storage_index = pd.MultiIndex.from_arrays([storage.carrier, storage.bus])
storage["p_min_pu"] = p_min_pu_stacked.reindex(storage_index).values 

#storage.to_csv('storage.csv', index=False)

storage.loc[storage.carrier=="battery", "p_min_pu"] = -1
storage.loc[storage.carrier=="hydro", "p_min_pu"] = 0
storage.loc[storage.carrier=="pondage", "p_min_pu"] = 0

storage = storage[storage.p_nom >0]

#storage.to_csv('storage.csv', index=False)

storage_inflows = inflows[storage.loc[storage.carrier=="PHS Open"].index]

#storage_inflows .to_csv('storage_inflows0.csv', index=False)

storage_inflows = pd.concat([storage_inflows,inflows[storage.loc[storage.carrier == "hydro"].index]],axis=1)
storage_inflows = pd.concat([storage_inflows,inflows[storage.loc[storage.carrier == "pondage"].index]],axis=1)

storage_inflows.to_csv('storage_inflows.csv', index=False)

hydro_stored = 1000000*(hydro_y.loc[hydro_y["PEMMDB_PLANT_TYPE"].isin(["Closed loop pumping", "Open loop pumping", "Reservoir", "Pondage"])]
  .pivot(index="PEMMDB_PLANT_TYPE", columns="MARKET_NODE", values="Storage Capacity [TWh]")
  .fillna(0)
  .rename(index={
      "Closed loop pumping": "PHS",
      "Open loop pumping": "PHS Open",
      "Reservoir": "hydro",
      "Pondage": "pondage"                    
  })
)       
  
battery = pd.read_csv("data/Dashboard_raw_data/Batteries additional information.csv",header=0)
battery = battery[battery.apply(lambda row: row.astype(str).str.contains("ERAA 2025 pre-CfE").any(), axis=1)]
battery.to_csv('battery.csv', index=False)

battery_y = battery[battery["TARGET_YEAR"] == base_year]

battery_y.to_csv('battery_y.csv', index=False)

battery_by_node = battery_y.groupby("MARKET_NODE")["STORAGE CAPACITY (MWh)"].sum().reset_index()

battery_by_node.to_csv('battery_y.csv', index=False)
battery_by_node["Technology"] = "battery"
battery_pivot = battery_by_node.pivot(index="Technology", columns="MARKET_NODE", values="STORAGE CAPACITY (MWh)").fillna(0)

battery_pivot.to_csv('battery_pivot.csv', index=False)

storage_capacity=pd.concat([hydro_stored, battery_pivot])

storage_capacity.to_csv('storage_capacity.csv', index=False)
  
valid_countries = storage_capacity.columns[(storage_capacity != 0).any(axis=0)]
stored_filtered = storage_capacity[valid_countries]
    
stored_stacked = stored_filtered.stack() 

storage_index = pd.MultiIndex.from_arrays([storage.carrier, storage.bus])

#Data in #.## TWh, so 5000 MWh as minimum
storage["max_hours"] = stored_stacked.reindex(storage_index).clip(lower=5000).div(storage.p_nom.values).values 
   
map_storage = pd.Series( ["battery inverter", "PHS", "hydro","hydro","hydro"], ["battery", "PHS", "PHS Open","hydro","pondage"])

storage["efficiency_store"] = technology_data.loc[:, "efficiency", :].reindex(storage.carrier.map(map_storage)).value.values

MARKET_NODE             AL00       AT00       BA00     BE00       BG00  \
PEMMDB_PLANT_TYPE                                                        
PHS                      0.0        0.0        0.0  10000.0    10000.0   
PHS Open                 0.0  1750000.0        0.0      0.0   390000.0   
pondage                  0.0    10000.0        0.0      0.0        0.0   
hydro              1720000.0   800000.0  1750000.0      0.0  1990000.0   

MARKET_NODE             CH00  CZ00      DE00        ES00       FI00  ...  \
PEMMDB_PLANT_TYPE                                                    ...   
PHS                  60000.0   0.0  600000.0    170000.0        0.0  ...   
PHS Open           1190000.0   0.0  440000.0   5840000.0        0.0  ...   
pondage                  0.0   0.0       0.0         0.0        0.0  ...   
hydro              8910000.0   0.0  240000.0  13340000.0  5530000.0  ...   

MARKET_NODE            RS00        SE01        SE02       SE03     SE04  SI00  \
PEMMDB_PLANT_TYPE

In [ ]:
    n.add(
        "StorageUnit",
        storage.index,
        **storage,
        inflow=storage_inflows.reindex(storage.index, axis=1, fill_value=0.),
        invest_status = "existing"
    )

In [ ]:
add_existing_storage(all_cap)

add_dispatchables()

add_renewables()

add_dsr()

set_investment_bounds()

n.add(
    "Link",
    links.index,
    bus0 = links.bus0,
    bus1 = links.bus1,
    p_nom = links.p_nom,
    p_max_pu = links_p_max_pu,
    carrier = links.carrier,
)

#n.generators.loc[(n.generators.bus == "CY00") & (n.generators.carrier == "CCGT"), "p_min_pu"] = 0 # remove minimum load of CCGT in Cyprus as this can exceed actual load.

dirname = os.path.dirname(save_path)  

if not os.path.exists(dirname):
    os.makedirs(dirname)

#n.export_to_netcdf(save_path)

/tmp/ipykernel_1749889/4291473729.py:13: FutureWarning:

DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.

/tmp/ipykernel_1749889/4291473729.py:14: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



MARKET_NODE             AL00       AT00       BA00     BE00       BG00  \
PEMMDB_PLANT_TYPE                                                        
PHS                      0.0        0.0        0.0  10000.0    10000.0   
PHS Open                 0.0  1750000.0        0.0      0.0   390000.0   
pondage                  0.0    10000.0        0.0      0.0        0.0   
hydro              1720000.0   800000.0  1750000.0      0.0  1990000.0   

MARKET_NODE             CH00  CZ00      DE00        ES00       FI00  ...  \
PEMMDB_PLANT_TYPE                                                    ...   
PHS                  60000.0   0.0  600000.0    170000.0        0.0  ...   
PHS Open           1190000.0   0.0  440000.0   5840000.0        0.0  ...   
pondage                  0.0   0.0       0.0         0.0        0.0  ...   
hydro              8910000.0   0.0  240000.0  13340000.0  5530000.0  ...   

MARKET_NODE            RS00        SE01        SE02       SE03     SE04  SI00  \
PEMMDB_PLANT_TYPE

ValueError: DataFrame inflow has an index which does not align with the passed names.

In [ ]:
commodity_prices_raw = pd.read_excel("data/Fuel and CO2 prices_ERAA2023.xlsx", index_col = 0)

year = 2029

climate_year = 1990

years = range(2025, 2034)

save_path = "resources/networks/base/cy" + str(climate_year) + "_ty" + str(year) + ".nc"

biomass_price = 7 # make adjustable in config

inflows_raw = pd.read_hdf("resources/inflow.h5", "inflow")

excel_file = pd.ExcelFile("data/pemmdb.xlsx")

snapshots = pd.date_range(start="2010-01-01", freq="h", periods=8760)

plant_sheets = [i for i in excel_file.sheet_names if "TY" in i]
plant_sheets = pd.Series([int(i[2:]) for i in plant_sheets], plant_sheets )

sheet = plant_sheets[plant_sheets <= year].subtract(year).idxmax()

base_year = int(sheet[2:])

distributed_resources = ["onwind", "offwind", "CSP","solar", "battery"]

In [25]:
demand = pd.read_hdf("resources/demand.h5")
demand = demand.loc[:, climate_year, :, str(base_year), :].unstack(1)
demand.index = snapshots
demand.drop("TR00",axis=1, inplace=True)

In [26]:
links = pd.read_hdf("resources/ntcs.h5", "p_nom")
links_p_max_pu = pd.read_hdf("resources/ntcs.h5", "p_max_pu")
links = links[base_year].unstack(1)
links_p_max_pu = links_p_max_pu[base_year].unstack(2)
links_p_max_pu.index = snapshots

In [27]:
links.dropna(inplace=True)
links_p_max_pu.dropna(axis=1, inplace=True)

In [28]:
links["carrier"] = [i[-2:] for i in links.index]

In [29]:
demand, links = group_luxembourg(demand, links)

In [30]:
commodity_prices = prepare_commodity_prices(commodity_prices_raw)

In [31]:
dispatchable_plants = pd.read_hdf("resources/dispatchable_capacities.h5")

In [32]:
technology_data = pd.read_csv("../technology-data/outputs/costs_2025.csv", index_col=[0,1])

In [33]:
n = pypsa.Network()
n.set_snapshots(snapshots)

In [34]:
capacity = build_capacity_table()

In [35]:
p_nom_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, :8]
marginal_cost_dsr = pd.read_excel(excel_file, "Explicit DSR", index_col = [0,1]).iloc[:, 8:16]

In [36]:
buses = (
    capacity.sum()[capacity.sum() >0].index
    .union(demand.columns)
    .union(links.bus0.unique())
    .union(links.bus1.unique())
)

In [37]:
n.madd(
    "Bus", 
    buses, 
    carrier = "electricity", 
    country = buses.str[:2]
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BEOF', 'BG00', 'CH00', 'CY00', 'CZ00',
       'DE00', 'DEKF', 'DKBH', 'DKE1', 'DKKF', 'DKW1', 'EE00', 'ES00', 'FI00',
       'FR00', 'GR00', 'GR03', 'HR00', 'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS',
       'ITN1', 'ITS1', 'ITSA', 'ITSI', 'ITVI', 'LT00', 'LUG1', 'LV00', 'ME00',
       'MK00', 'MT00', 'NL00', 'NLLL', 'NOM1', 'NON1', 'NOS0', 'PL00', 'PLE0',
       'PLI0', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04', 'SI00',
       'SK00', 'UK00', 'UKNI'],
      dtype='object')

In [38]:
n.madd(
    "Load", 
    demand.columns,
    bus=demand.columns,
    p_set = demand
)

Index(['AL00', 'AT00', 'BA00', 'BE00', 'BG00', 'CH00', 'CY00', 'CZ00', 'DE00',
       'DKE1', 'DKW1', 'EE00', 'ES00', 'FI00', 'FR00', 'GR00', 'GR03', 'HR00',
       'HU00', 'IE00', 'ITCA', 'ITCN', 'ITCS', 'ITN1', 'ITS1', 'ITSA', 'ITSI',
       'LT00', 'LUG1', 'LV00', 'ME00', 'MK00', 'MT00', 'NL00', 'NOM1', 'NON1',
       'NOS0', 'PL00', 'PT00', 'RO00', 'RS00', 'SE01', 'SE02', 'SE03', 'SE04',
       'SI00', 'SK00', 'UK00', 'UKNI'],
      dtype='object', name='zone')

In [39]:
inflows = build_inflows(inflows_raw)

In [40]:
add_existing_storage()

In [41]:
add_dispatchables()

/tmp/ipykernel_2335640/1052378219.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plants["marginal_cost"] = (


In [42]:
add_renewables()

In [43]:
add_dsr()

In [44]:
set_investment_bounds()

In [45]:
n.madd(
    "Link",
    links.index,
    bus0 = links.bus0,
    bus1 = links.bus1,
    p_nom = links.p_nom,
    p_max_pu = links_p_max_pu,
    carrier = links.carrier
)

Index(['AL00-GR00-AC', 'AL00-ME00-AC', 'AL00-MK00-AC', 'AL00-RS00-AC',
       'AT00-CH00-AC', 'AT00-CZ00-AC', 'AT00-DE00-AC', 'AT00-HU00-AC',
       'AT00-ITN1-AC', 'AT00-SI00-AC',
       ...
       'UK00-BE00-DC', 'UK00-DE00-DC', 'UK00-DKW1-DC', 'UK00-FR00-DC',
       'UK00-IE00-DC', 'UK00-NL00-DC', 'UK00-NOS0-DC', 'UK00-UKNI-DC',
       'UKNI-IE00-AC', 'UKNI-UK00-DC'],
      dtype='object', length=212)

In [46]:
n.madd(
    "Generator",
    n.buses.index + " load-shedding",
    carrier = "load-shedding",
    bus = n.buses.index,
    p_nom=1e5,
    marginal_cost = 10e3,
)

Index(['AL00 load-shedding', 'AT00 load-shedding', 'BA00 load-shedding',
       'BE00 load-shedding', 'BEOF load-shedding', 'BG00 load-shedding',
       'CH00 load-shedding', 'CY00 load-shedding', 'CZ00 load-shedding',
       'DE00 load-shedding', 'DEKF load-shedding', 'DKBH load-shedding',
       'DKE1 load-shedding', 'DKKF load-shedding', 'DKW1 load-shedding',
       'EE00 load-shedding', 'ES00 load-shedding', 'FI00 load-shedding',
       'FR00 load-shedding', 'GR00 load-shedding', 'GR03 load-shedding',
       'HR00 load-shedding', 'HU00 load-shedding', 'IE00 load-shedding',
       'ITCA load-shedding', 'ITCN load-shedding', 'ITCS load-shedding',
       'ITN1 load-shedding', 'ITS1 load-shedding', 'ITSA load-shedding',
       'ITSI load-shedding', 'ITVI load-shedding', 'LT00 load-shedding',
       'LUG1 load-shedding', 'LV00 load-shedding', 'ME00 load-shedding',
       'MK00 load-shedding', 'MT00 load-shedding', 'NL00 load-shedding',
       'NLLL load-shedding', 'NOM1 load-shedding', 

In [49]:
n.storage_units.efficiency_store

StorageUnit
AL00 reservoir    1.0
AT00 PHS          1.0
AT00 reservoir    1.0
AT00 battery      1.0
BA00 reservoir    1.0
                 ... 
SK00 reservoir    1.0
SK00 battery      1.0
UK00 PHS          1.0
UK00 battery      1.0
UKNI battery      1.0
Name: efficiency_store, Length: 89, dtype: float64

In [1]:
dirname = os.path.dirname(save_path)  

if not os.path.exists(dirname):
    os.makedirs(dirname)

NameError: name 'os' is not defined

In [ ]:
n.optimize(snapshots=n.snapshots[:24], solver_name="highs")